In [ ]:
from music21 import *
from comp2 import *

In [ ]:
melody_in: stream.Stream = parse_midi("example_inputs/eyeofthetiger.mid")

print("Original melody:")
melody_in.show('midi');

In [ ]:
print("Simple triads:")
simple_triads = get_triads(melody_in)

In [ ]:
out_score = stream.Score()
out_score.insert(0, melody_in)
out_score.insert(0, simple_triads)

out_score.show('midi')

In [ ]:
markov_table = generate_markov_table()
substituted_chords = substitute_chords(simple_triads, markov_table)
for i in range(5):
    substituted_chords = substitute_chords(substituted_chords, markov_table)

out_score = stream.Score()
out_score.insert(0, melody_in)
out_score.insert(0, substituted_chords)

out_score.show('midi')

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, MultiHeadAttention, Dense, Layer, LayerNormalization, Dropout
from tensorflow.keras.models import Model

In [ ]:
class Encoder(Layer):
    def __init__(self, num_layers, d_model, num_heads, dff):
        super().__init__()
        self.layers = [
            TransformerBlock(  # Custom layer
                d_model, num_heads, dff,
                bidirectional_attention=True  # ← Key modification
            ) for _ in range(num_layers)
        ]
    
    def call(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
class JazzTransformer(Model):
    def __init__(self, num_layers=6, d_model=512, num_heads=8, dff=2048):
        super().__init__()
        self.encoder = Encoder(num_layers, d_model, num_heads, dff)
        self.decoder = Decoder(num_layers, d_model, num_heads, dff)
        self.forward_head = Dense(token_vocab_size)
        self.backward_head = Dense(token_vocab_size)
        
    def call(self, inputs):
        # Input shape: (batch_size, seq_len, d_model)
        enc_output = self.encoder(inputs)
        dec_output = self.decoder(enc_output)
        
        return {
            "forward": self.forward_head(dec_output),
            "backward": self.backward_head(dec_output)
        }